In [1]:
"""
The `CreateUnPack` module only exports the `@create_unpack` macro,
which creates a macro unpacking fields or properties of an object.

__Simple Intended Usage:__
```
julia> using .CreateUnPack

julia> struct Foo{A, B, C} a::A; b::B; c::C end

julia> @create_unpack all_Foo Foo
@unpackall_Foo (macro with 1 method)

help?> @unpackall_Foo
@unpackall_Foo(x) unpacks the fields (:a, :b, :c) of x.

julia> @macroexpand @unpackall_Foo foo
:((a, b, c) = (foo.a, foo.b, foo.c))

julia> @unpackall_Foo Foo(1, 2.0, "three")
(1, 2.0, "three")

julia> a, b, c
(1, 2.0, "three")
```
"""
module CreateUnPack

export @create_unpack

"""
    @create_unpack(name::Symbol, expr)

creates the macro named `Symbol(:unpack, name)`
which unpacks the fields specified by `expr` of an object.

Let `val` be the value of `expr`.

* If `val` is a tuple of symbols, then set the unpacking fields to `val`.
* If `val` is a typp, then set the unpacking fields to `fieldnames(val)`.
* Otherwise, set the unpacking fields to `propertynames(val)`.

__Example:__

```
julia> @create_unpack _cat_and_dog (:cat, :dog)
@unpack_cat_and_dog (macro with 1 method)

help?> @unpack_cat_and_dog
@unpack_cat_and_dog(obj) unpacks the fields (:cat, :dog) of obj.

julia> @macroexpand @unpack_cat_and_dog x
:((cat, dog) = (x.cat, x.dog))

julia> @unpack_cat_and_dog (dog = "bowwow", mouse = "squeak", cat = "meow")
("meow", "bowwow")

julia> cat, dog
("meow", "bowwow")

julia> struct Foo{A, B, C} a::A; b::B; c::C end

julia> @create_unpack all_Foo Foo
@unpackall_Foo (macro with 1 method)

help?> @unpackall_Foo
@unpackall_Foo(obj) unpacks the fields (:a, :b, :c) of obj.

julia> @macroexpand @unpackall_Foo foo
:((a, b, c) = (foo.a, foo.b, foo.c))

julia> @unpackall_Foo Foo(1, 2.0, "three")
(1, 2.0, "three")

julia> a, b, c
(1, 2.0, "three")

julia> baz = (p = "one", q = 2.0, r = 3)

julia> @create_unpack all_baz baz
@unpackall_baz (macro with 1 method)

help?> @unpackall_baz
@unpackall_baz(obj) unpacks the fields (:p, :q, :r) of obj.

julia> @macroexpand @unpackall_baz baz
:((p, q, r) = (baz.p, baz.q, baz.r))

julia> @unpackall_baz baz
("one", '2', 3)

julia> p, q, r
("one", '2', 3)

```
"""
macro create_unpack(name::Symbol, expr)
    macroname = Symbol(:unpack, name)
    atmacroname = Symbol('@', macroname)
    val = Core.eval(__module__, expr)
    names = val isa Tuple{Vararg{Symbol}} ? val :
            val isa Type ? fieldnames(val) : propertynames(val)
    docstr = """`$atmacroname(obj)` unpacks the fields `$names` of `obj`."""
    quote
        macro $macroname(obj)
            Expr(:(=),
                Expr(:tuple, $names...),
                Expr(:tuple, (:($obj.$name) for name in $names)...)
            ) |> esc
        end
        @doc $docstr $(:($atmacroname))
        $atmacroname
    end |> esc
end

end

Main.CreateUnPack

In [2]:
@doc CreateUnPack

The `CreateUnPack` module only exports the `@create_unpack` macro, which creates a macro unpacking fields or properties of an object.

**Simple Intended Usage:**

```
julia> using .CreateUnPack

julia> struct Foo{A, B, C} a::A; b::B; c::C end

julia> @create_unpack all_Foo Foo
@unpackall_Foo (macro with 1 method)

help?> @unpackall_Foo
@unpackall_Foo(x) unpacks the fields (:a, :b, :c) of x.

julia> @macroexpand @unpackall_Foo foo
:((a, b, c) = (foo.a, foo.b, foo.c))

julia> @unpackall_Foo Foo(1, 2.0, "three")
(1, 2.0, "three")

julia> a, b, c
(1, 2.0, "three")
```


In [3]:
using .CreateUnPack
@doc @create_unpack

```
@create_unpack(name::Symbol, expr)
```

creates the macro named `Symbol(:unpack, name)` which unpacks the fields specified by `expr` of an object.

Let `val` be the value of `expr`.

  * If `val` is a tuple of symbols, then set the unpacking fields to `val`.
  * If `val` is a typp, then set the unpacking fields to `fieldnames(val)`.
  * Otherwise, set the unpacking fields to `propertynames(val)`.

**Example:**

```
julia> @create_unpack _cat_and_dog (:cat, :dog)
@unpack_cat_and_dog (macro with 1 method)

help?> @unpack_cat_and_dog
@unpack_cat_and_dog(obj) unpacks the fields (:cat, :dog) of obj.

julia> @macroexpand @unpack_cat_and_dog x
:((cat, dog) = (x.cat, x.dog))

julia> @unpack_cat_and_dog (dog = "bowwow", mouse = "squeak", cat = "meow")
("meow", "bowwow")

julia> cat, dog
("meow", "bowwow")

julia> struct Foo{A, B, C} a::A; b::B; c::C end

julia> @create_unpack all_Foo Foo
@unpackall_Foo (macro with 1 method)

help?> @unpackall_Foo
@unpackall_Foo(obj) unpacks the fields (:a, :b, :c) of obj.

julia> @macroexpand @unpackall_Foo foo
:((a, b, c) = (foo.a, foo.b, foo.c))

julia> @unpackall_Foo Foo(1, 2.0, "three")
(1, 2.0, "three")

julia> a, b, c
(1, 2.0, "three")

julia> baz = (p = "one", q = 2.0, r = 3)

julia> @create_unpack all_baz baz
@unpackall_baz (macro with 1 method)

help?> @unpackall_baz
@unpackall_baz(obj) unpacks the fields (:p, :q, :r) of obj.

julia> @macroexpand @unpackall_baz baz
:((p, q, r) = (baz.p, baz.q, baz.r))

julia> @unpackall_baz baz
("one", '2', 3)

julia> p, q, r
("one", '2', 3)

```


In [4]:
@create_unpack _cat_and_dog (:cat, :dog)

@unpack_cat_and_dog (macro with 1 method)

In [5]:
?@unpack_cat_and_dog

`@unpack_cat_and_dog(obj)` unpacks the fields `(:cat, :dog)` of `obj`.


In [6]:
@macroexpand @unpack_cat_and_dog x

:((cat, dog) = (x.cat, x.dog))

In [7]:
@unpack_cat_and_dog (dog = "bowwow", mouse = "squeak", cat = "meow")

("meow", "bowwow")

In [8]:
cat, dog

("meow", "bowwow")

In [9]:
struct Foo{A, B, C} a::A; b::B; c::C end

In [10]:
@create_unpack all_Foo Foo

@unpackall_Foo (macro with 1 method)

In [11]:
?@unpackall_Foo

`@unpackall_Foo(obj)` unpacks the fields `(:a, :b, :c)` of `obj`.


In [12]:
@macroexpand @unpackall_Foo foo

:((a, b, c) = (foo.a, foo.b, foo.c))

In [13]:
@unpackall_Foo Foo(1, 2.0, "three")

(1, 2.0, "three")

In [14]:
a, b, c

(1, 2.0, "three")

In [15]:
function f(foo::Foo)
    @unpackall_Foo foo
    @show a b c
    a, b, c
end

f (generic function with 1 method)

In [16]:
f(Foo(1, 2.0, "three"))

a = 1
b = 2.0
c = "three"


(1, 2.0, "three")

In [17]:
baz = (p = "one", q = 2.0, r = 3)

(p = "one", q = 2.0, r = 3)

In [18]:
@create_unpack all_baz baz

@unpackall_baz (macro with 1 method)

In [19]:
?@unpackall_baz

`@unpackall_baz(obj)` unpacks the fields `(:p, :q, :r)` of `obj`.


In [20]:
@macroexpand @unpackall_baz baz

:((p, q, r) = (baz.p, baz.q, baz.r))

In [21]:
@unpackall_baz baz

("one", 2.0, 3)

In [22]:
p, q, r

("one", 2.0, 3)

In [23]:
module A
using ..CreateUnPack
struct Bar{X, Y} x::X; y::Y end
@create_unpack Bar Bar
end

Main.A

In [24]:
?A.@unpackBar

`@unpackBar(obj)` unpacks the fields `(:x, :y)` of `obj`.


In [25]:
@macroexpand A.@unpackBar bar

:((x, y) = (bar.x, bar.y))

In [26]:
A.@unpackBar A.Bar(1, 2)

(1, 2)

In [27]:
x, y

(1, 2)